# Factor Construction 

From the Intelligent Optimization Based Multi-Factor Deep Learning Stock Selection Model and Quantitative Trading Strategy and Factor Selection with Deep Reinforcement Learning for Financial Forecasting, I select several technical and financial factors to construct. The following table shows the factor constructed. 



| Factors | Shortname | Description/ Formula | Category |
|--------|------|-------------|----------|
| 20 day annualized return variance | V20 | Variance of annualized returns of individual stocks in the last 20 days | Technical |
| 20 day return kurtosis | K20 | 20 day kurtosis of individual stock returns | Technical |
| 10 day average turnover rate | TURN10 | Average turnover rate of individual stocks in the last 10 days | Technical |
| Moving average of 12 day trading volume | VEMA20 | Moving average of 12 day trading volume of individual stocks | Technical |
| Standard deviation of 20 day trading volume | VSTD20 | Standard deviation of trading volume of individual stocks in the last 20 days | Technical |
| Buying and selling momentum rate | AR | (sum of 26 days (highest price of the day – opening price of the day) / sum of 26 days (opening price of the day – lowest price of the day) * 100% | Technical |
| Willingness to buy rate | BR | (sum of 26 days (the highest price of the day - yesterday’s closing price) / sum of 26 days (yesterday’s closing price – the lowest price of the day) * 100% | Technical |
| Arron up | AU | ((calculation period days – days after the highest price) / calculation period days) * 100% | Technical |
| Arron down | AD | ((calculation period days – days after the lowest price) / calculation period days) * 100% | Technical |
| Net profit margin on sales     | NPR          | net profit / operating income                     | Financial |
| Retained earnings per share    | RPPS         | retained profit / closing share capital           | Financial |
| Altman Z Score| Altman_Z_Score | 1.2A+1.4B +3.3C+0.6D+ 0.999e (A = working capital / total assets, B = retained earnings / total assets, C = earnings before interest and tax / total assets, D = market value of equity / total liabilities,E = sales / total assets) | Financial |


### Data and Package 

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
ratio = pd.read_pickle('../data/ratio.pkl')

In [16]:
price = pd.read_pickle('../data/prices.pkl')

In [18]:
price_sorted = price.groupby('ticker', group_keys=False).apply(lambda x: x.sort_values('date'))


In [19]:
price_sorted

adj_close    adj_open    adj_high     adj_low  \
ticker date                                                                  
A      2000-01-03 00:00:00   46.867298   51.261107   51.384785   43.859980   
       2000-01-04 00:00:00   43.287157   44.348181   44.836382   42.148021   
       2000-01-05 00:00:00   40.071540   43.124423   43.163479   39.908806   
       2000-01-06 00:00:00   39.056082   40.117105   40.357951   37.838834   
       2000-01-07 00:00:00   42.310755   38.444203   42.922634   38.405147   
...                                ...         ...         ...         ...   
ZTS    2024-02-21 00:00:00  188.380000  186.660000  188.860000  186.660000   
       2024-02-22 00:00:00  196.660000  189.310000  196.670000  188.540000   
       2024-02-23 00:00:00  197.210000  197.350000  198.210000  194.815000   
       2024-02-26 00:00:00  197.040000  196.900000  197.990000  195.460000   
       2024-02-27 00:00:00  199.000000  197.650000  199.250000  194.040000   

                               volume  
ticker date                            
A      2000-01-03 00:00:00  3343600.0  
       2000-01-04 00:00:00  3408500.0  
       2000-01-05 00:00:00  4119200.0  
       2000-01-06 00:00:00  1812900.0  
       2000-01-07 00:00:00  2016900.0  
...                               ...  
ZTS    2024-02-21 00:00:00  3178767.0  
       2024-02-22 00:00:00  3339537.0  
       2024-02-23 00:00:00  2389058.0  
       2024-02-26 00:00:00  2086579.0  
       2024-02-27 00:00:00  1971765.0  

[2090573 rows x 5 columns]

In [20]:
ratio

curr_ratio  free_cash_flow_per_share  ret_equity  \
date       ticker                                                     
2000-01-31 A              NaN                       NaN         NaN   
           AAPL           NaN                       NaN         NaN   
           ABT            NaN                       NaN         NaN   
           ADBE           NaN                       NaN         NaN   
           ADI            NaN                       NaN         NaN   
...                       ...                       ...         ...   
2024-02-29 XYL         1.7222                    0.8547    1.233796   
           YUM         1.2600                    4.6246   -5.589829   
           ZBH         1.6132                    4.6676    3.236602   
           ZBRA        1.0463                   -1.7598    0.554926   
           ZTS         3.3579                    3.5153   10.369553   

                   gross_margin  ret_asset  tot_debt_tot_equity  \
date       ticker                                                 
2000-01-31 A                NaN        NaN                  NaN   
           AAPL             NaN        NaN                  NaN   
           ABT              NaN        NaN                  NaN   
           ADBE             NaN        NaN                  NaN   
           ADI              NaN        NaN                  NaN   
...                         ...        ...                  ...   
2024-02-29 XYL          36.8015   0.458395             0.184222   
           YUM          73.7230   0.878650            -1.351611   
           ZBH          72.2217   1.185020             0.444830   
           ZBRA         44.4004   0.092227             0.724704   
           ZTS          67.1035   0.518507             1.302070   

                   oper_profit_margin  invty_turn  ret_invst  \
date       ticker                                              
2000-01-31 A                      NaN         NaN        NaN   
           AAPL                   NaN         NaN        NaN   
           ABT                    NaN         NaN        NaN   
           ADBE                   NaN         NaN        NaN   
           ADI                    NaN         NaN        NaN   
...                               ...         ...        ...   
2024-02-29 XYL                 9.2004      1.2148   1.024250   
           YUM                29.9116     47.8000  13.537415   
           ZBH                18.2474      0.2259   2.342744   
           ZBRA                7.3340      0.6978   0.331804   
           ZTS                29.3719      0.2839   4.482128   

                   pretax_profit_margin  ...   ret_0_1   ret_1_6  ret_6_12  \
date       ticker                        ...                                 
2000-01-31 A                        NaN  ...       NaN       NaN       NaN   
           AAPL                     NaN  ...       NaN       NaN       NaN   
           ABT                      NaN  ...       NaN       NaN       NaN   
           ADBE                     NaN  ...       NaN       NaN       NaN   
           ADI                      NaN  ...       NaN       NaN       NaN   
...                                 ...  ...       ...       ...       ...   
2024-02-29 XYL                   8.9114  ... -0.110202 -0.120878  0.023306   
           YUM                  22.7898  ... -0.056831 -0.011927 -0.019248   
           ZBH                  14.6650  ... -0.042009 -0.049816  0.056825   
           ZBRA                  0.1982  ... -0.083603  0.054795  0.113616   
           ZTS                  29.3719  ... -0.036390 -0.039319 -0.094372   

                       skew  volume_shock  rs_volatility  p_volatility  \
date       ticker                                                        
2000-01-31 A            NaN      0.604983       0.023066      0.021375   
           AAPL         NaN      0.897883       0.036539      0.033523   
           ABT          NaN      1.008005       0.008837      0.007463   
     

### Technical 

In [21]:
def calculate_technical_factors(price): 
    technical_factor = pd.DataFrame()
    price['daily_return'] = price.groupby('ticker')['adj_close'].pct_change()
    technical_factor['V20']= price.groupby('ticker')['daily_return'].rolling(window=20).var() * np.sqrt(252)
    technical_factor['K20']= price.groupby('ticker')['daily_return'].rolling(window=20).kurt()
    price['turnover'] = price.groupby('ticker')['volume'].pct_change() + 1
    technical_factor['TURN10'] = price.groupby('ticker')['turnover'].rolling(window=10).mean()
    technical_factor['VEMA12'] = price.groupby('ticker')['volume'].rolling(window=12).mean() 
    technical_factor['VSTD20'] = price.groupby('ticker')['volume'].rolling(window=20).std()
    price['high_open'] = price['adj_high'] - price['adj_open']
    price['open_low'] = price['adj_open'] - price['adj_low']
    price['previous_close'] = price.groupby('ticker')['adj_close'].shift(1)
    price['high_prev_close'] = price['adj_high'] - price['previous_close']
    price['prev_close_low'] = price['previous_close'] - price['adj_low']
    technical_factor['AR'] = price.groupby('ticker')['high_open'].rolling(window=26).sum() / price.groupby('ticker')['open_low'].rolling(window=26).sum() * 100
    technical_factor['BR'] = price.groupby('ticker')['high_prev_close'].rolling(window=26).sum() / price.groupby('ticker')['prev_close_low'].rolling(window=26).sum() * 100
    
    def days_since_high(x):
        return np.argmax(x[::-1])

    def days_since_low(x):
        return np.argmin(x[::-1])

    days_since_high = price.groupby('ticker')['adj_high'].rolling(window=10).apply(days_since_high, raw=True)
    days_since_low = price.groupby('ticker')['adj_low'].rolling(window=10).apply(days_since_low, raw=True)

    days_since_high = days_since_high.reset_index(level=0, drop=True)
    days_since_low = days_since_low.reset_index(level=0, drop=True)

    price['days_since_high'] = days_since_high
    price['days_since_low'] = days_since_low

    technical_factor.index = price.index

    technical_factor['AU'] = (10 - price['days_since_high']) / 10 * 100
    technical_factor['AD'] = (10 - price['days_since_low']) / 10 * 100

    #grouped = technical_factor.groupby(level='ticker')
    #resampled_groups = {ticker: group.resample('M', level='date').last() for ticker, group in grouped}

    #resampled_df = pd.concat(resampled_groups)

    #resampled_df = resampled_df.sort_index()

    #return resampled_df
    return technical_factor

In [22]:
df1 = calculate_technical_factors(price_sorted)

### Financial 

| Net profit margin on sales     | NPR          | net profit / operating income                     | Financial |
| Retained earnings per share    | RPPS         | retained profit / closing share capital           | Financial |
| Enterprise Multiple     | EBITDA/EV         | nep_val/ EBITDA| Financial |
| Altman Z Score| Altman_Z_Score | 1.2A+1.4B +3.3C+0.6D+ 0.999e (A = working capital / total assets
B = retained earnings / total assets
C = earnings before interest and tax / total assets
D = market value of equity / total liabilities
E = sales / total assets) | Financial |

The Altman Z Score formula is (1.2 x A) + (1.4 x B) + (3.3 x C) + (0.6 x D) + (0.999 x E). 

A = working capital / total assets
B = retained earnings / total assets
C = earnings before interest and tax / total assets
D = market value of equity / total liabilities
E = sales / total assets

NPR = NET_INCOME_LOSS_SHARE_HOLDER/ OPER_INCOME

RPPS = RETAIN_EARN_ACCUM_DEFICIT/ TOT_LIAB_SHARE_HOLDER_EQUITY

EBITDA/EV = ep_val/ EBITDA

working capital = TOT_CURR_ASSET - TOT_CURR_LIAB
Altman_Z_Score = 1.2 * ( workingcapital/	TOT_ASSET) + 1.4*(	RETAIN_EARN_ACCUM_DEFICIT/TOT_ASSET ) + 3.3*(EBIT/TOT_ASSET) + (mkt_val/TOT_LIAB) + (gross_margin/TOT_ASSET)

In [23]:
zack = pd.read_pickle('../data/zack.pkl')

In [24]:
zack

per_end_date   mkt_val officer_title_1  treas_stock  \
ticker date                                                       
A      NaT    2023-09-30  32717.11             NaN          NaN   
       NaT    2023-06-30  35518.94             NaN          NaN   
       NaT    2023-03-31  40907.39             NaN          NaN   
       NaT    2022-12-31  44275.46             NaN          NaN   
       NaT    2022-09-30  35983.73             NaN          NaN   
...                  ...       ...             ...          ...   
ZTS    NaT    2007-03-31       NaN             NaN          NaN   
       NaT    2006-12-31       NaN             NaN          NaN   
       NaT    2006-09-30       NaN             NaN          NaN   
       NaT    2006-06-30       NaN             NaN          NaN   
       NaT    2006-03-31       NaN             NaN          NaN   

             incr_decr_cash  eps_basic_acct_change  tot_non_oper_income_exp  \
ticker date                                                                   
A      NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
...                     ...                    ...                      ...   
ZTS    NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   
       NaT              NaN                    NaN                      NaN   

             defer_revnu_curr  cash_flow_oper_activity_qd  \
ticker date                                                 
A      NaT                NaN                         NaN   
       NaT                NaN                         NaN   
       NaT                NaN                         NaN   
       NaT                NaN                         NaN   
       NaT                NaN                         NaN   
...                       ...                         ...   
ZTS    NaT                NaN                         NaN   
       NaT                NaN                         NaN   
       NaT                NaN                         NaN   
       NaT                NaN                         NaN   
       NaT                NaN                         NaN   

             comm_stock_div_paid  ...  form_7_type rcv_turn  ret_asset  \
ticker date                       ...                                    
A      NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
...                          ...  ...          ...      ...        ...   
ZTS    NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   
       NaT                   NaN  ...          NaN      NaN        NaN   

             oper_cash_flow_per_share  oper_profit_margin  asset_turn  \
ticker date                                                             
A      NaT                        NaN                 NaN         NaN   
       NaT                        NaN                 NaN         NaN   
       NaT                        NaN                 NaN      

In [25]:
def calculate_financial_factors(zack):
    finacial_indicator = pd.DataFrame() 
    zack = zack.reset_index()
    zack['per_end_date'] = pd.to_datetime(zack['per_end_date'])
    zack['date'] = zack['date'].fillna(zack['per_end_date'])
    zack.sort_values(by=['ticker', 'date'], inplace=True)
    zack = zack.set_index(['ticker', 'date'])
    zack = zack.groupby('ticker').ffill() 
    display(zack)
    finacial_indicator['NPR'] = zack['net_income_loss_share_holder']/zack['oper_income']
    finacial_indicator['RPPS'] = zack['retain_earn_accum_deficit']/zack['tot_liab_share_holder_equity']
    finacial_indicator['working capital'] = zack['tot_curr_asset'] - zack['tot_curr_liab']
    finacial_indicator['A'] = finacial_indicator['working capital']/zack['tot_asset']
    finacial_indicator['B'] = zack['retain_earn_accum_deficit']/zack['tot_asset']
    finacial_indicator['C'] = zack['ebit']/zack['tot_asset']
    finacial_indicator['D'] = zack['mkt_val']/zack['tot_liab']
    finacial_indicator['E'] = zack['gross_margin']/zack['tot_asset']
    finacial_indicator['Altman_Z_Score'] = 1.2* finacial_indicator['A']+1.4* finacial_indicator['B']+3.3* finacial_indicator['C']+0.6* finacial_indicator['D']+0.999* finacial_indicator['E']

    finacial_indicator = finacial_indicator.drop(['A', 'B', 'C', 'D', 'E','working capital'], axis=1)
    return finacial_indicator

In [26]:
df2 = calculate_financial_factors(zack) 

per_end_date   mkt_val  \
ticker date                                
A      2006-03-10   2006-01-31       NaN   
       2006-03-31   2006-03-31  16099.94   
       2006-06-07   2006-04-30  16099.94   
       2006-06-30   2006-06-30  13672.45   
       2006-09-07   2006-07-31  13672.45   
...                        ...       ...   
ZTS    2023-02-15   2022-12-31  68302.85   
       2023-05-05   2023-03-31  76913.95   
       2023-08-09   2023-06-30  79580.33   
       2023-11-03   2023-09-30  79876.60   
       2024-02-14   2023-12-31  90615.27   

                                        officer_title_1  treas_stock  \
ticker date                                                            
A      2006-03-10                                   NaN       3281.0   
       2006-03-31                                   NaN       3281.0   
       2006-06-07                                   NaN       3768.0   
       2006-06-30                                   NaN       3768.0   
       2006-09-07                                   NaN       4469.0   
...                                                 ...          ...   
ZTS    2023-02-15  Chief Executive Officer and Director       4539.0   
       2023-05-05  Chief Executive Officer and Director       4807.0   
       2023-08-09  Chief Executive Officer and Director       5126.0   
       2023-11-03  Chief Executive Officer and Director       5369.0   
       2024-02-14  Chief Executive Officer and Director       5597.0   

                   incr_decr_cash  eps_basic_acct_change  \
ticker date                                                
A      2006-03-10           511.0                    NaN   
       2006-03-31           511.0                    NaN   
       2006-06-07           431.0                    NaN   
       2006-06-30           431.0                    NaN   
       2006-09-07            23.0                    NaN   
...                           ...                    ...   
ZTS    2023-02-15            96.0                    NaN   
       2023-05-05         -1472.0                    NaN   
       2023-08-09         -1864.0                    NaN   
       2023-11-03         -1827.0                    NaN   
       2024-02-14         -1540.0                    NaN   

                   tot_non_oper_income_exp  defer_revnu_curr  \
ticker date                                                    
A      2006-03-10                     48.0             262.0   
       2006-03-31                     48.0             262.0   
       2006-06-07                     46.0             293.0   
       2006-06-30                     46.0             293.0   
       2006-09-07                     45.0             292.0   
...                                    ...               ...   
ZTS    2023-02-15                     -8.0               NaN   
       2023-05-05                     -8.0               NaN   
       2023-08-09                     -8.0               NaN   
       2023-11-03                     -8.0               NaN   
       2024-02-14                     -8.0               NaN   

                   cash_flow_oper_activity_qd  comm_stock_div_paid  ...  \
ticker date                                                         ...   
A      2006-03-10                      -128.0                  NaN  ...   
       2006-03-31                      -128.0                  NaN  ...   
       2006-06-07                       296.0                -16.0  ...   
       2006-06-30                       296.0                -16.0  ...   
       2006-09-07                        65.0                -16.0  ...   
...                                       ...                  ...  ...   
ZTS    2023-02-15                       741.0               -611.0  ...   
       2023-05-05                       549.0               -174.0  ...   
       2023-08-09                       183.0               -347.0  ...   
       2023-11-03                       724.0               -520.0  .

In [27]:
df1.reset_index(inplace=True)
df1['date'] = pd.to_datetime(df1['date'])
df1.set_index(['ticker', 'date'], inplace=True)

In [28]:
df2

NPR      RPPS  Altman_Z_Score
ticker date                                           
A      2006-03-10  80.457143  0.168577             NaN
       2006-03-31  80.457143  0.168577        3.216715
       2006-06-07   1.173469  0.182202        3.142772
       2006-06-30   1.173469  0.182202        2.777094
       2006-09-07   1.247253  0.219304        2.869934
...                      ...       ...             ...
ZTS    2023-02-15   0.778716  0.580771        5.205956
       2023-05-05   0.791966  0.657627        6.470039
       2023-08-09   0.769495  0.694087        6.816390
       2023-11-03   0.831241  0.718843        6.891655
       2024-02-14   0.807692  0.720636        7.400778

[32705 rows x 3 columns]

In [29]:
df1 = df1.reset_index()
df2 = df2.reset_index()

In [30]:
df1

,ticker,date,V20,K20,TURN10,VEMA12,VSTD20,AR,BR,AU,AD
0,A,2000-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A,2000-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A,2000-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A,2000-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A,2000-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2090568,ZTS,2024-02-21,0.006677,5.769814,1.077293,2.485598e+06,658415.268671,107.434037,82.851039,40.0,50.0
2090569,ZTS,2024-02-22,0.008043,4.516172,1.098362,2.588333e+06,698330.830311,133.336978,110.389782,30.0,40.0
2090570,ZTS,2024-02-23,0.008031,4.487588,1.095837,2.587854e+06,696585.174917,132.201373,116.843378,20.0,30.0
2090571,ZTS,2024-02-26,0.007698,4.825829,1.033304,2.594122e+06,690596.142760,133.992554,112.524020,10.0,20.0


In [31]:
merged_df = pd.merge(df1, df2, on=['ticker', 'date'], how='inner')

In [32]:
factor = merged_df.set_index(['ticker', 'date'])

In [34]:
factor = factor.dropna()

In [35]:
factor 

V20       K20    TURN10        VEMA12        VSTD20  \
ticker date                                                                   
A      2006-03-31  0.001677 -0.161433  1.009188  2.677283e+06  8.507770e+05   
       2006-06-07  0.008970  4.639936  1.011806  3.033917e+06  1.593372e+06   
       2006-06-30  0.008720 -0.337261  1.298395  2.861767e+06  1.582068e+06   
       2006-09-07  0.012354  4.094771  1.084502  2.034808e+06  1.145483e+06   
       2007-06-06  0.004510  4.969473  0.958086  2.506725e+06  1.468824e+06   
...                     ...       ...       ...           ...           ...   
ZTS    2023-02-15  0.005827  0.623090  1.009657  2.374945e+06  7.575722e+05   
       2023-05-05  0.002627  3.730305  1.154390  1.989963e+06  8.547849e+05   
       2023-08-09  0.007405  3.522361  1.047530  2.342736e+06  9.595190e+05   
       2023-11-03  0.006718  5.146277  1.145770  2.644201e+06  1.469757e+06   
       2024-02-14  0.006188  6.098433  1.091327  2.162124e+06  7.283716e+05   

                           AR          BR     AU     AD        NPR      RPPS  \
ticker date                                                                    
A      2006-03-31  150.897694   99.348513   10.0   70.0  80.457143  0.168577   
       2006-06-07   66.018349   53.477136   70.0   20.0   1.173469  0.182202   
       2006-06-30   86.024845   64.109589   10.0   50.0   1.173469  0.182202   
       2006-09-07  117.285221  198.396764   70.0  100.0   1.247253  0.219304   
       2007-06-06  260.060060  160.086768   10.0   30.0   0.938931  0.248112   
...                       ...         ...    ...    ...        ...       ...   
ZTS    2023-02-15   68.499006  136.146193   90.0   70.0   0.778716  0.580771   
       2023-05-05  149.641497  146.003099  100.0   90.0   0.791966  0.657627   
       2023-08-09  148.895864  158.482527   10.0   90.0   0.769495  0.694087   
       2023-11-03   63.419721   64.695119   10.0   80.0   0.831241  0.718843   
       2024-02-14   95.384921   79.232211   80.0   90.0   0.807692  0.720636   

                   Altman_Z_Score  
ticker date                        
A      2006-03-31        3.216715  
       2006-06-07        3.142772  
       2006-06-30        2.777094  
       2006-09-07        2.869934  
       2007-06-06        2.986211  
...                           ...  
ZTS    2023-02-15        5.205956  
       2023-05-05        6.470039  
       2023-08-09        6.816390  
       2023-11-03        6.891655  
       2024-02-14        7.400778  

[21569 rows x 12 columns]

In [200]:
factor.to_pickle("factor_new_construct_z.pkl")